In [18]:
import os
import torch
import numpy as np
import pandas as pd
from rdkit import Chem
from pathlib import Path
from rdkit.Chem import SDMolSupplier
from torch_geometric.data import InMemoryDataset, Data


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/kelvin/miniconda3/envs/jupyter/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/kelvin/miniconda3/envs/jupyter/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/kelvin/miniconda3/envs/jupyter/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/kelvin/miniconda3/envs/jupyter/lib/python3.10/site-packages/traitlets/config/application.

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/kelvin/miniconda3/envs/jupyter/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/kelvin/miniconda3/envs/jupyter/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/kelvin/miniconda3/envs/jupyter/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/kelvin/miniconda3/envs/jupyter/lib/python3.10/site-packages/traitlets/config/application.

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/kelvin/miniconda3/envs/jupyter/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/kelvin/miniconda3/envs/jupyter/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/kelvin/miniconda3/envs/jupyter/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/kelvin/miniconda3/envs/jupyter/lib/python3.10/site-packages/traitlets/config/application.

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/kelvin/miniconda3/envs/jupyter/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/kelvin/miniconda3/envs/jupyter/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/kelvin/miniconda3/envs/jupyter/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/kelvin/miniconda3/envs/jupyter/lib/python3.10/site-packages/traitlets/config/application.

AttributeError: _ARRAY_API not found

In [32]:
def featurize_rdkit_mol(mol):  
    num_atoms = mol.GetNumAtoms()
    node_feats = torch.randn(num_atoms, 23)
    edge_index = torch.rand(2, 8)
    edge_attr = torch.rand(8, 5)
    pos = torch.rand(40, 3)

    return {
        "x": node_feats,
        "edge_index": edge_index,
        "edge_attr": edge_attr,
        "pos": pos
    }

In [35]:
class PDBBindLigandDataset(InMemoryDataset):
    def __init__(self, metadata_csv="data/processed/refined_dataset_metadata.csv", 
                 root="data/processed/graphs", transform=None, 
                 pre_transform=None, force_rebuild=False):
        self.metadata_csv = Path(metadata_csv)
        self.root = Path(root)
        self.root.mkdir(parents=True, exist_ok=True)
        self.cache_file = self.root / "dataset.pt"
        self.force_rebuild = force_rebuild
        super().__init__(self.root, transform, pre_transform)
        if self.cache_file.exists() and not self.force_rebuild:
            self.data, self.slices = torch.load(self.cache_file)
        else:
            self.process()
            self.data, self.slices = torch.load(self.cache_file)

    @property
    def raw_file_names(self):
        return [self.metadata_csv.name]

    @property
    def processed_file_names(self):
        return [self.cache_file.name]

    def process(self):
        df = pd.read_csv(self.metadata_csv)
        data_list = []
        skipped = 0
        for idx, row in df.iterrows():
            cid = row["complex_id"]
            ligand_path = Path(row["ligand_file"])
            affinity = float(row["affinity"])
            if not ligand_path.exists():
                skipped += 1
                continue
                
            mol = None
            if ligand_path.suffix.lower() == ".sdf":
                supplier = SDMolSupplier(str(ligand_path), removeHs=False)
                mol = supplier[0]
            else:
                mol = Chem.MolFromMolFile(str(ligand_path), removeHs=False)

            if mol is None:
                skipped += 1
                continue

            feats = featurize_rdkit_mol(mol)
            if feats is None:
                skipped += 1
                continue

            data = Data(x = feats["x"], edge_index = feats["edge_index"],
                edge_attr = feats["edge_attr"], pos = feats["pos"] if feats["pos"].shape[0] > 0 else None,
                y = torch.tensor([affinity], dtype=torch.float32), complex_id = cid)
            data_list.append(data)

        print(f"done processing. total processed: {len(data_list)}. skipped: {skipped}")
        if len(data_list) == 0:
            raise RuntimeError("no data processed")

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.cache_file)
        print(f"saved processed dataset to {self.cache_file}")

    def get(self, idx):
        return super().get(idx)

    def len(self):
        return super().len()


In [36]:
dataset = PDBBindLigandDataset(metadata_csv="data/processed/refined_dataset_metadata.csv", root="data/processed/graphs", force_rebuild=False)

Processing...
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Can't kekulize mol.  Unkekulized atoms: 9 10 11 12 13
[14:42:20] ERROR: Could not sanitize molecule ending on line 52
[14:42:20] ERROR: Can't kekulize mol.  Unkekulized atoms: 9 10 11 12 13
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[

[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Can't kekulize mol.  Unkekulized atoms: 11 12 13 14 15
[14:42:20] ERROR: Could not sanitize molecule ending on line 66
[14:42:20] ERROR: Can't kekulize mol.  Unkekulized atoms: 11 12 13 14 15
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Wa

[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Can't kekulize mol.  Unkekulized atoms: 26 27 28 31 35
[14:42:20] ERROR: Could not sanitize molecule ending on line 121
[14:42:20] ERROR: Can't kekulize mol.  Unkekulized atoms: 26 27 28 31 35
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] W

[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Can't kekulize mol.  Unkekulized atoms: 13 14 15 16 17 18 19 20 21
[14:42:20] ERROR: Could not sanitize molecule ending on line 217
[14:42:20] ERROR: Can't kekulize mol.  Unkekulized atoms: 13 14 15 16 17 18 19 20 21
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the

[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 18 19 20 22 23
[14:42:20] ERROR: Could not sanitize molecule ending on line 141
[14:42:20] ERROR: Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 18 19 20 22 23
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:20] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 

[14:42:21] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:21] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:21] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:21] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:21] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:21] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:21] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:21] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:21] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol

[14:42:21] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:21] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 10 11 12 13
[14:42:21] ERROR: Could not sanitize molecule ending on line 98
[14:42:21] ERROR: Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 10 11 12 13
[14:42:21] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:21] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:21] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:21] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:21] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:21] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as

[14:42:21] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:21] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:21] Can't kekulize mol.  Unkekulized atoms: 4 5 8 9 10
[14:42:21] ERROR: Could not sanitize molecule ending on line 82
[14:42:21] ERROR: Can't kekulize mol.  Unkekulized atoms: 4 5 8 9 10
[14:42:21] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:21] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:21] Can't kekulize mol.  Unkekulized atoms: 15 16 17 18 19 20 22 23 24
[14:42:21] ERROR: Could not sanitize molecule ending on line 115
[14:42:21] ERROR: Can't kekulize mol.  Unkekulized atoms: 15 16 17 18 19 20 22 23 24
[14:42:21] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[14:42:21] Can't 

done processing. total processed: 4640. skipped: 676
saved processed dataset to data/processed/graphs/dataset.pt


Done!


In [31]:
dataset[0]

tensor([[ 1.1252,  0.3324,  2.5643, -1.3343, -0.2427, -0.0847,  0.5269, -0.7784,
          0.3021, -0.4475, -0.5100,  0.8162, -0.5990, -0.1138,  0.7163,  0.9841,
          0.0417,  1.5528,  0.7071,  0.4826,  0.1713,  1.0584, -0.7446],
        [-1.0334, -0.1417, -2.2066,  2.0162, -0.6491, -0.6376,  0.1786,  1.2968,
         -0.5536, -1.4659, -1.5914, -1.3039, -0.8880,  0.6915,  0.6436, -0.2079,
         -0.4292,  0.7731, -0.4019,  0.2691,  0.0402,  0.7506, -1.2884],
        [ 0.0235,  2.0812, -0.2429, -0.9752,  0.2847, -1.5462,  0.8913, -1.0681,
          1.7903, -0.8840, -2.1741,  1.0755, -0.0107,  0.6381,  0.8520,  3.6226,
         -0.5990, -0.0281, -0.6095,  0.5255, -0.7872,  0.2542,  2.3898],
        [ 0.4519, -1.2453, -0.9353, -0.0670, -1.2103, -0.6890,  0.7027,  1.4316,
         -1.4888,  0.3881, -0.1844, -1.4064,  0.9444, -0.4537, -1.3205,  0.3719,
         -1.9240, -0.0210,  0.8592, -0.9180,  0.6635,  0.2340,  0.4259],
        [-0.5030,  2.6556, -0.0942, -1.2403, -1.0221,  0.250